In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplhep as hep
from iminuit import Minuit
from iminuit.cost import LeastSquares
import pickle
import uproot as ur
import re

In [2]:
input_file = f"../../genIII_data/Sim/positron_4GeV.edm4hep.root"

events = ur.open("%s:events"%(input_file))

with open("sim_geometry.pkl", 'rb') as file:
    sim_geometry = pickle.load(file)
    

even_ch_map = sim_geometry["even_ch_map"]
odd_ch_map = sim_geometry["odd_ch_map"]
z_to_layer_map = sim_geometry["z_to_layer_map"]

In [3]:
num_evts = len(np.array(events["HcalFarForwardZDCHits"]["HcalFarForwardZDCHits.energy"]))
all_event_energies = []

for evt in range(num_evts):
    if evt % 100 == 0:
        print(100 * evt / num_evts)
    evt_energies = []

    energies = np.array(events["HcalFarForwardZDCHits"]["HcalFarForwardZDCHits.energy"])[evt]
    x_pos    = np.array(events["HcalFarForwardZDCHits"]["HcalFarForwardZDCHits.position.x"])[evt]
    y_pos    = np.array(events["HcalFarForwardZDCHits"]["HcalFarForwardZDCHits.position.y"])[evt]
    z_pos    = np.array(events["HcalFarForwardZDCHits"]["HcalFarForwardZDCHits.position.z"])[evt]

    # Get layer numbers for each hit
    layers = np.array([z_to_layer_map[z] for z in z_pos])

    for layer in range(15):
        # Select channel map based on even/odd layer
        ch_map = even_ch_map if layer % 2 == 0 else odd_ch_map

        for ch in range(25):
            # Find (x, y) pair for this channel
            try:
                xy_lookup = [k for k, v in ch_map.items() if v == ch][0]  # (x, y)
            except IndexError:
                # If this channel index doesn't exist in the map, append 0
                evt_energies.append(0)
                continue

            # Mask for hits at this layer and this (x, y) position
            mask = (layers == layer) & (x_pos == xy_lookup[0]) & (y_pos == xy_lookup[1])
            
            if np.any(mask):
                # If there's a matching hit, get the energy (assume only one per channel)
                energy = energies[mask][0]
            else:
                energy = 0

            evt_energies.append(energy/0.000875127161931984)

    all_event_energies.append(evt_energies)



0.0
10.0
20.0
30.0
40.0
50.0
60.0
70.0
80.0
90.0


In [4]:
# Invert the channel maps: ch → (x, y)
even_ch_to_xy = {v: k for k, v in even_ch_map.items()}
odd_ch_to_xy  = {v: k for k, v in odd_ch_map.items()}

# Invert the z-layer map: layer → z
layer_to_z = {v: float(k) for k, v in z_to_layer_map.items()}

positions = []

for layer in range(15):
    ch_map = even_ch_to_xy if layer % 2 == 0 else odd_ch_to_xy

    for ch in range(25):
        if ch in ch_map and layer in layer_to_z:
            x, y = ch_map[ch]
            z = layer_to_z[layer]
        else:
            x, y, z = None, None, None
        positions.append([x, y, z-5002.756, layer, ch])


In [5]:
sim_data = {
    'positions': np.array(positions),
    'events': np.array(all_event_energies, dtype=object)
}

In [6]:
with open('../../genIII_data/Sim/Positrons.pkl', 'wb') as f:
    pickle.dump(sim_data, f)